In [ ]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import json
from scipy.stats import pearsonr, spearmanr

from sklearn.model_selection import train_test_split
from pandas import DataFrame
from datetime import datetime
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats
from sklearn.svm import SVR
import joblib
import lightgbm as lgb

In [ ]:
dataset = pd.read_csv('/kaggle/input/new-data-14052025/DATASET-ECFP-FS-100 (2).csv')
#data = dataset.iloc[:,1:]
data = dataset
data

In [ ]:
rata_rata = data['pIC50'].mean()

print("Rata-rata nilai:", rata_rata)

In [ ]:
x = data.drop(columns=['pIC50'])
y = data['pIC50']

In [ ]:
x

In [ ]:
train_ratio = 0.70
#validation_ratio = 0.20
test_ratio = 0.30

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size = 1- train_ratio, random_state=42)
#x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=42) 

In [ ]:
x_train

In [ ]:
"""
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')
"""

In [ ]:
print(x_train.shape)

In [ ]:

regressor = xgb.XGBRegressor(
    learning_rate= 0.01,
    n_estimators=500,
    max_depth=20,
    subsample= 0.5,
)

regressor.fit(x_train, y_train,)


In [ ]:
"""
regressor = SVR(
    C=10,
    epsilon=0.2,
    gamma= 'scale',
    kernel='rbf'
)

regressor.fit(x_train, y_train)
"""

In [ ]:
"""
regressor = lgb.LGBMRegressor(
    colsample_bytree=0.8,
    learning_rate=0.1,
    max_depth=-1,
    n_estimators=500,
    num_leaves=127,
    subsample=0.6,
    )

regressor.fit(x_train, y_train)
"""

In [ ]:
""""
evals_result = regressor.evals_result()
regressor.save_model('xgb_model_ecfp_4.json')

with open('training_logs_ecfp_4.json', 'w') as f:
    json.dump(evals_result, f)
"""

In [ ]:
joblib.dump(regressor, 'model.pkl')

In [ ]:
prediction = regressor.predict(x_test)
output = pd.DataFrame(prediction)

In [ ]:
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    mean_absolute_percentage_error, median_absolute_error,
    explained_variance_score
)

In [ ]:
print(f"MSE  : {mean_squared_error(y_test, output):.4f}")
print(f"RMSE : {mean_squared_error(y_test, output, squared=False):.4f}")
print(f"MAE  : {mean_absolute_error(y_test, output):.4f}")
print(f"R²   : {r2_score(y_test, output):.4f}")
print(f"MAPE : {mean_absolute_percentage_error(y_test, output):.4f}")
print(f"MedAE: {median_absolute_error(y_test, output):.4f}")
print(f"EVS  : {explained_variance_score(y_test, output):.4f}")

In [ ]:
y_true = np.array(y_test)
y_pred = np.array(output)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_true, y_pred, color='skyblue', label='Data Asli vs Prediksi')
plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red', linestyle='--', label='Garis Identitas')

plt.title('Scatter Plot: Data Asli vs Prediksi')
plt.xlabel('Data Asli')
plt.ylabel('Prediksi')

# Menambahkan legenda
plt.legend()

# Menampilkan plot
plt.show()

# Klasifikasi

In [ ]:
output_df = pd.DataFrame(output).rename(columns={0:"pIC50"})
output_df

In [ ]:
y_test_df = pd.DataFrame(y_test).rename(columns={0:"pIC50"})
y_test_df

In [ ]:
def classify_ic50(ic50):
    if ic50 < 1:
        return 'Aktivitas sangat baik atau poten'
    elif 1 <= ic50 <= 20:
        return 'Aktivitas baik atau sitotoksisitas sangat kuat'
    elif 20 < ic50 <= 100:
        return 'Aktivitas sedang'
    elif 100 < ic50 <= 200:
        return 'Aktivitas rendah'
    else:
        return 'Tidak aktif'

In [ ]:
output_df['IC50_uM'] = 10 ** (-output_df['pIC50']) * 1e6
output_df['Kriteria'] = output_df['IC50_uM'].apply(classify_ic50)
output_df

In [ ]:
y_test_df['IC50_uM'] = 10 ** (-y_test_df['pIC50']) * 1e6
y_test_df['Kriteria'] = y_test_df['IC50_uM'].apply(classify_ic50)
y_test_df

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

pred = output_df['Kriteria']
test = y_test_df['Kriteria']

accuracy = accuracy_score(test, pred)
precision = precision_score(test, pred, average='weighted')
recall = recall_score(test, pred, average='weighted')
f1 = f1_score(test, pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Confidence Interval

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

In [ ]:
"""
scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'rmse': make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'r2': make_scorer(r2_score),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False),
    'medae': make_scorer(median_absolute_error, greater_is_better=False),
    'evs': make_scorer(explained_variance_score)
}
"""

In [ ]:
"""
cv_results = cross_validate(regressor, x_train, y_train, cv=5, scoring=scoring)

# Fungsi hitung CI 95%
def ci_95(data):
    mean = np.mean(data)
    std = np.std(data, ddof=1)
    n = len(data)
    t_val = stats.t.ppf(1 - 0.025, df=n - 1)
    return mean, mean - t_val * (std / np.sqrt(n)), mean + t_val * (std / np.sqrt(n))

# Hasil
for metric in scoring.keys():
    values = -cv_results[f'test_{metric}'] if metric in ['mse', 'rmse', 'mae', 'mape', 'medae'] else cv_results[f'test_{metric}']
    mean, lower, upper = ci_95(values)
    print(f"{metric.upper()} Mean: {mean:.4f}, 95% CI: ({lower:.4f}, {upper:.4f})")

"""

# Feature Importance

In [ ]:

from sklearn.inspection import permutation_importance

# Contoh data dan model regresi
X_importance = x
y_importance = y
model = xgb.XGBRegressor()
model.fit(X_importance, y_importance)

# Hitung permutation importance
result = permutation_importance(model, X_importance, y_importance, n_repeats=10, random_state=42)

# Membuat DataFrame untuk melihat hasilnya
importance_df = pd.DataFrame({
    'Feature': X_importance.columns,
    'Importance': result.importances_mean
}).sort_values(by='Importance', ascending=False)

print(importance_df)



In [ ]:
importance_df.to_csv('feature_importance.csv', index=False)